In [2]:
import os
import numpy as np

In [218]:
def file_to_array(filename):
    loc = os.path.join("..", "data", filename)
    with open(loc) as f:
        data = f.read()
    return np.array([list(line) for line in data.split("\n")])


data = file_to_array("day12.txt")

In [219]:
surroundings = [
    np.array((-1, 0)),
    np.array((0, -1)),
    np.array((1, 0)),
    np.array((0, 1)),
]


def check_outside_bounds(test_position, size):
    return (
        test_position[0] < 0
        or test_position[1] < 0
        or test_position[0] >= size[0]
        or test_position[1] >= size[1]
    )


def need_fence(loc1, loc2, data):
    if check_outside_bounds(loc2, data.shape):
        return True
    return data[*loc1] != data[*loc2]


def expand_region(current_region, data):
    old_region = []
    new_region = current_region.copy()
    newly_discovered = []
    while new_region:
        for loc in new_region:
            surrounding_areas = [tuple(area) for area in np.array(loc) + surroundings]
            for next_area in surrounding_areas:
                if (
                    next_area not in new_region
                    and next_area not in old_region
                    and next_area not in newly_discovered
                    and not need_fence(loc, next_area, data)
                ):
                    newly_discovered.append(next_area)

        old_region.extend(new_region)
        new_region = newly_discovered.copy()
        newly_discovered = []
    return old_region

In [220]:
regions = []

discovered_areas = []
for x in range(data.shape[1]):
    for y in range(data.shape[0]):
        current_area = (y, x)
        if current_area in discovered_areas:
            continue
        current_region = [current_area]
        current_region = expand_region(current_region, data)

        regions.append(current_region)
        discovered_areas.extend(current_region)
len(regions)

594

In [174]:
costs = []
for region in regions:
    # placements = np.argwhere(data == crop)
    area = len(region)
    placements = np.array(region)
    perimeter = 0
    for loc in placements:
        surrounding_areas = loc + surroundings
        perimeter += sum(
            need_fence(loc, next_area, data) for next_area in surrounding_areas
        )
    costs.append(perimeter * area)

In [ ]:
costs = []
for region in regions:
    # placements = np.argwhere(data == crop)
    area = len(region)
    placements = np.array(region)
    perimeter = 0
    for loc in placements:
        surrounding_areas = loc + surroundings
        perimeter += sum(
            need_fence(loc, next_area, data) for next_area in surrounding_areas
        )
    costs.append(perimeter * area)

In [175]:
sum(costs)

1467094

# Part 2

In [316]:
def get_starting_point(region):
    region = np.array(region)
    top_row = np.min(region[:, 0])
    left_most_in_top_row = np.min(region[region[:, 0] == top_row, 1])
    return (top_row, left_most_in_top_row)


def change_direction(current_direction):
    if (current_direction == np.array([-1, 0])).all():
        return np.array([0, 1])
    elif (current_direction == np.array([0, 1])).all():
        return np.array([1, 0])
    elif (current_direction == np.array([1, 0])).all():
        return np.array([0, -1])
    elif (current_direction == np.array([0, -1])).all():
        return np.array([-1, 0])
    else:
        raise ValueError("Wrong direction")


def index_direction(direction):
    direction_index = {
        (-1, 0): 0,
        (0, 1): 1,
        (1, 0): 2,
        (0, -1): 3,
    }  # indexing numpy arrays doesnt work
    direction = tuple(direction)
    return direction_index[direction]


def extra_sides(new_direction, old_direction):
    index_diff = (
        index_direction(new_direction) + 4 - index_direction(old_direction)
    ) % 4
    if index_diff == 2:
        return 2
    if index_diff in (1, 3):
        return 1
    if index_diff == 0:
        return 0


def step(position, direction, region):
    # Maximum exploration
    desired_direction = directions[index_direction(direction) - 1]
    direction = desired_direction.copy()

    while True:
        test_position = tuple(position + direction)
        if test_position in region:
            return test_position, direction

        direction = change_direction(direction)
        if (direction == desired_direction).all():
            raise ValueError("Can not escape!")


def get_n_sides(region):
    states = []
    starting_point = get_starting_point(region)
    sides = 0
    position = starting_point
    # Way to get starting point and starting directions match!
    original_direction = np.array([0, 1])
    direction = original_direction.copy()
    while True:
        try:
            new_position, new_direction = step(position, direction, region)
        except ValueError:
            return 4  # can not escape from a one shape

        if not states and index_direction(new_direction) == 2:
            # This fence gets added in the last step
            sides += 0
        else:
            sides += extra_sides(new_direction, direction)
        position = new_position
        direction = new_direction

        state = (position, tuple(direction))
        if state in states:
            return sides
        states.append(state)
        step(position, direction, region)


def get_sides(fence, horizontal=True):
    sides = len(fence)
    for piece in fence:
        if horizontal and (piece[0], piece[1] + 1) in fence:
            sides -= 1
        if not horizontal and (piece[0] + 1, piece[1]) in fence:
            sides -= 1
    return sides


def region_surrounded_by_other_region(region, surrounding_region):
    for loc in region:
        for direction in surroundings:
            next_area = tuple(loc + direction)
            if next_area in region or next_area in surrounding_region:
                continue
            return False
    return True

In [312]:
def file_to_array(filename):
    loc = os.path.join("..", "data", filename)
    with open(loc) as f:
        data = f.read()
    return np.array([list(line) for line in data.split("\n")])


data = file_to_array("day12.txt")

In [313]:
regions = []

discovered_areas = []
for x in range(data.shape[1]):
    for y in range(data.shape[0]):
        current_area = (y, x)
        if current_area in discovered_areas:
            continue
        current_region = [current_area]
        current_region = expand_region(current_region, data)

        regions.append(current_region)
        discovered_areas.extend(current_region)
len(regions)

594

In [314]:
# costs = []
# for region in regions:
#     horizontal_fence = []
#     vertical_fence = []

#     area = len(region)
#     placements = np.array(region)
#     for loc in placements:
#         for direction in surroundings:
#             next_area = loc + direction
#             if need_fence(loc, next_area, data):
#                 if (direction == np.array([-1, 0])).all():
#                     horizontal_fence.append(tuple(loc))
#                 if (direction == np.array([1, 0])).all():
#                     fence_loc = tuple(loc)
#                     fence_loc = (fence_loc[0] + 1, fence_loc[1])
#                     horizontal_fence.append(fence_loc)
#                 if (direction == np.array([0, -1])).all():
#                     vertical_fence.append(tuple(loc))
#                 if (direction == np.array([0, 1])).all():
#                     fence_loc= tuple(loc)
#                     fence_loc = (fence_loc[0], fence_loc[1] + 1)
#                     vertical_fence.append(fence_loc)
#     sides = get_sides(vertical_fence, False) + get_sides(horizontal_fence, True)
#     costs.append(sides * area)
# print(costs)
# print(sum(costs))

In [315]:
region_costs = {}
for id_, region in enumerate(regions):
    area = len(region)
    n_sides = get_n_sides(region)
    region_costs[id_] = {"area": area, "outside_sides": n_sides}

for id_, region in enumerate(regions):

    region_costs[id_]["sides"] = region_costs[id_]["outside_sides"]
    for id_other, other_region in enumerate(regions):
        if region_surrounded_by_other_region(other_region, region):
            region_costs[id_]["sides"] += region_costs[id_other]["outside_sides"]

costs = [values["area"] * values["sides"] for _, values in region_costs.items()]
costs

[448,
 860,
 160,
 8760,
 340,
 6048,
 4,
 8,
 156,
 12432,
 2698,
 760,
 864,
 5880,
 4,
 16,
 1764,
 16,
 810,
 7344,
 5500,
 2464,
 6960,
 2610,
 5184,
 4,
 4,
 4,
 4,
 8,
 8,
 8,
 4,
 4,
 2688,
 18,
 182,
 3154,
 4,
 4,
 5704,
 4,
 4,
 4,
 18,
 4,
 4,
 16,
 18,
 4,
 4,
 400,
 3996,
 1230,
 7700,
 4,
 4,
 624,
 288,
 24,
 1350,
 520,
 8,
 6832,
 24,
 4,
 4,
 8,
 22746,
 4,
 18,
 216,
 2278,
 256,
 18,
 3306,
 24,
 112,
 876,
 4,
 6844,
 4,
 294,
 4,
 18,
 2492,
 896,
 2142,
 2952,
 5876,
 4,
 216,
 180,
 30,
 4,
 4,
 682,
 4,
 8844,
 24,
 4,
 4,
 3552,
 4,
 30,
 4920,
 17190,
 8,
 8,
 70,
 738,
 924,
 30,
 24,
 3792,
 4,
 8,
 4356,
 4,
 2898,
 4,
 60,
 8,
 5152,
 4,
 12,
 120,
 3872,
 96,
 18,
 1534,
 16,
 7070,
 8,
 676,
 648,
 1120,
 4,
 4,
 1792,
 4,
 18,
 6642,
 4,
 4,
 5356,
 238,
 18,
 378,
 56,
 8,
 12,
 5400,
 4,
 4,
 2584,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 7980,
 8,
 1472,
 512,
 4,
 4,
 2808,
 108,
 16200,
 2814,
 18,
 30,
 6664,
 3550,
 5324,
 1890,
 532,
 8,
 4,
 2898,
 4

In [317]:
sum(costs)

902026